In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# http://scikit-image.org/docs/stable/api/api.html
import skimage

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://matplotlib.org/api/index.html
import matplotlib

# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt

import statistics 
from statistics import mode 

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
IMG_SHAPE = (64, 64)
ROOT_PATH = os.getcwd()

h5_det = "model_data_aug_64_64_gpu_detector.h5"
h5_cla = "model_data_aug_64_64_bel_gpu.h5"

activation_det = "elu"
activation_cla = "selu"

opt_det = "adam"
opt_cla = "rmsprop"

labels_path = os.path.join(ROOT_PATH, "dataset_bel/labels.csv")
titles = traffic.read_csv(labels_path, ",")

# Images to test
images_test_path = [os.path.join(ROOT_PATH, "images_test/Prohibido_001.jpg"),
                    os.path.join(ROOT_PATH, "images_test/Rotonda_001.jpg"),
                    os.path.join(ROOT_PATH, "images_test/Stop_001.jpg")]

In [3]:
def get_keras_model(classes, activation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=activation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(classes, activation='softmax'))
    return model

In [4]:
def load_trained_model_detector(weights_path):
    model = get_keras_model(2, activation_det)
    model.load_weights(weights_path)
    return model

def load_trained_model_classificator(weights_path, classes):
    model = get_keras_model(classes, activation_cla)
    model.load_weights(weights_path)
    return model

In [5]:
model_detector = load_trained_model_detector(os.path.join(ROOT_PATH, h5_det))
model_detector.compile(loss="categorical_crossentropy", optimizer=opt_det, metrics=['accuracy'])

model_classificator = load_trained_model_classificator(os.path.join(ROOT_PATH, h5_cla), len(titles))
model_classificator.compile(loss="categorical_crossentropy", optimizer=opt_cla, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
def get_tiles(image, tile_shape, offset):
    tiles = []
    xx = tile_shape[0]
    yy = tile_shape[1]
    for x in range(0, image.shape[0]- xx, offset):
        for y in range(0, image.shape[1] - yy, offset):
            tile = image[x:x + xx, y:y + yy]
            tiles.append(tile)
    return tiles

In [7]:
def get_normalized_image(image_path):
    image_test = skimage.data.imread(image_path)
    height = image_test.shape[0]
    width = image_test.shape[1]
    return skimage.transform.resize(image_test, (height, width), mode='constant')

In [8]:
signs = []
for image_path in images_test_path:
    image_test = get_normalized_image(image_path)
    image_tiles = get_tiles(image_test, IMG_SHAPE, 1)
    signs_aux = []
    for tile in image_tiles:
        np_tile = np.expand_dims(tile, axis=0)
        result = model_detector.predict_classes(np_tile)
        if result == 1:
            result_sign = model_classificator.predict_classes(np_tile)
            signs_aux.append(result_sign[0])
    print("Tiles: {} - valid tiles: {} Moda: {} Sign {}".format(len(image_tiles), len(signs_aux), mode(signs_aux), titles[mode(signs_aux)]))
    signs.append(signs_aux)

Tiles: 60976 - valid tiles: 7588 Moda: 22 Sign ['22', 'C1']
Tiles: 60976 - valid tiles: 5750 Moda: 35 Sign ['35', 'D1b']
Tiles: 61182 - valid tiles: 4048 Moda: 22 Sign ['22', 'C1']
